In [1]:
# nbi:hide_in
##########################
# Created on Mar 2020
# @author: juans
##########################

In [2]:
# nbi:hide_in
# Requried libraries
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
%matplotlib inline

In [3]:
# nbi:hide_in
def QuantizeUniform(aNum,nBits):
    #Uniformly quantize signed fraction aNum with nBits
    #Notes:The overload level of the quantizer should be 1.0    
    if(aNum >= 0):
        s = 0
    else:
        s = 1
    if(np.abs(aNum) >= 1):
        code = 2**(nBits-1)-1
    else:
        code = int(((2**(nBits)-1)*np.abs(aNum)+1)/2)
    if(s == 0):
       return int(code)
    else:
       return int(code + 2**(nBits-1))
   
def DequantizeUniform(aQuantizedNum,nBits):
    #Uniformly dequantizes nBits-long number aQuantizedNum 
    #into a signed fraction
    if (aQuantizedNum>>(nBits-1)):
        sign = -1
    else:
        sign = 1
    if (sign == -1):
        aNum =  sign*float(2*np.abs(aQuantizedNum - 2**(nBits-1)))/(2**nBits -1)    
    else:
        aNum =  sign*float(2*np.abs(aQuantizedNum))/(2**nBits -1) 
    return aNum

def ScaleFactor(aNum, nScaleBits, nMantBits):
    # Return the floating-point scale factor for a  signed 
    # fraction aNum given nScaleBits scale bits and nMantBits mantissa bits  
    #Notes:
    #The scale factor should be the number of leading zeros
    if nScaleBits < 0: nScaleBits = 0
    if nMantBits <= 0: return 0       # 0 mantissa bits returns 0
    
    maxScale=(1 << nScaleBits) - 1    # max leading 0s scale can id
    maxBits = maxScale + nMantBits    # max bits equiv to this FP rep
    signBit = (1 << (maxBits - 1))    # the location of the sign bit in the Uni Q num
    
    # Uniformly quantize magnitue using maxBits and left shift away sign bit
    code = QuantizeUniform(abs(aNum), maxBits)
    code <<= 1
    # get Scale by shifting left till you hit a 1
    scale = 0
    while scale < maxScale and (signBit & code) == 0:
        code <<= 1
        scale += 1
    
    return scale

def MantissaFP(aNum, scale, nScaleBits, nMantBits):
    # Return the floating-point mantissa for a  signed fraction 
    # aNum given nScaleBits scale bits and nMantBits mantissa bits
    if nMantBits <= 0: return 0
    if nScaleBits < 0: nScaleBits = 0
        
    maxScale = (1 << nScaleBits) - 1
    maxBits = maxScale + nMantBits
    signBit = (1 << (nMantBits - 1))
    
    # Extract sign
    sign = 0
    if aNum < 0:
        sign = 1
        aNum *= -1
        
    # Compute unsigned code using maxBits uniform quantization
    code = QuantizeUniform(aNum, maxBits)
    # extract the mantissa: shift left by scale factor and sign (remove leading 0)
    code <<= (scale + 1)
    # remove leading 1 (if we know it is there) and shfit Left on emore time
    if scale < maxScale:
        code -= (1 << (maxBits - 1)) 
        code <<= 1
    # move bits starting at maxBits down to loest nMantBits - 1
    code >>= (maxBits - nMantBits + 1)
    # add sign to the front of the code
    if sign: code += signBit
        
    return code

def DequantizeFP(scale, mantissa, nScaleBits, nMantBits):
    # Returns a signed fraction for floating point scale and mantissa given
    # Specified scale and mantissa bits
    # zero mantissa bits means zero
    if nMantBits <= 0: return 0
    if nScaleBits < 0: nScaleBits = 0
        
    maxScale = (1 << nScaleBits ) - 1
    maxBits = maxScale + nMantBits
    signBit = (1 << (nMantBits - 1))
    
    if mantissa & signBit:
        sign = 1
        mantissa -= signBit
    else:
        sign = 0
    
    if scale < maxScale:
        mantissa = mantissa + (1 << (nMantBits - 1))
        
    if scale < (maxScale - 1):
        mantissa = (mantissa << 1) + 1
        mantissa <<= (maxScale - scale - 2)
        
    if sign:
        signBit = (1 << (maxBits -1))
        mantissa += signBit
        
    return DequantizeUniform(mantissa, maxBits)

In [4]:
# nbi:hide_in
nScaleBitsSlider = widgets.IntSlider(
    value=3,
    min=2,
    max=4,
    step=1,
    description='nScaleBits',
    continuous_update=False,
    readout_format='d',
)

nMantBitsSlider = widgets.IntSlider(
    value=3,
    min=2,
    max=8,
    step=1,
    description='nMantBits',
    continuous_update=False,
    readout_format='d',
)

In [5]:
# nbi:hide_in
# Interactive plot generation to exemplify different quantizers
def plot(nScaleBits, nMantBits):
    
    # Create a vector of amplitudes with at least 4 points per
    # Quantization step
    nTotBits = nScaleBits + nMantBits;
    x = np.linspace(-1, 1, 2**nTotBits * 4)
    # Array for Quantized codes
    y = np.zeros_like(x, dtype=int)
    # Array for Dequantized float values
    z = np.zeros_like(x)

    for i in np.arange(len(y)):
        s = ScaleFactor(x[i], nScaleBits, nMantBits)
        m = MantissaFP(x[i], s, nScaleBits, nMantBits)
        z[i] = DequantizeFP(s, m, nScaleBits, nMantBits)

    plt.plot(x, z, '.')
    plt.axis('equal')
    plt.axis('square')
    plt.xlim([-1, 1])
    plt.ylim([-1, 1])
    plt.xticks(plt.yticks()[0], rotation=90)
    plt.grid(True)
    plt.xlabel('Continuous Amplitude')
    plt.ylabel('Quantized Amplitude')
    plt.title('Floating Point Quantizer')
    plt.tight_layout()
    plt.show();

# Execute interaction
widgets.interactive(plot, nScaleBits=nScaleBitsSlider, nMantBits=nMantBitsSlider)

interactive(children=(IntSlider(value=3, continuous_update=False, description='nScaleBits', max=4, min=2), Int…